# Seminario 1 de Python: Gestión de memoria.

Python es un **lenguaje orientado a objetos**. En un primer momento esto no resulta muy aparente, ya que hasta ahora hemos utilizado un estilo de programación *imperativo*. No obstante, aún sin ser completamente conscientes de ello, realmente hemos estado creando y manejando objetos en nuestros programas. En Python absolutamente *todo* lo que se manipula son objetos: números, cadenas, listas, tuplas, diccionarios, funciones, módulos, etc.

La mayoría de los lenguajes orientados a objetos, incluyendo Python, cuentan con una gestión dinámica de memoria integrada en el mismo *runtime* del lenguaje, lo cual permite ordenar la creación y destrucción de objetos durante la ejecución de los programas.

Python, gracias a tratarse de un lenguaje interpretado, es además un **lenguaje dinámico**: así, algunos objetos se crean bajo demanda, es decir, en el mismo instante en que se necesitan, sin necesidad de crearlos explícitamente, ni de especificar su tipo/clase, que Python determina a partir del modo en que se utilizan dichos objetos. Así mismo, los objetos que quedan sin uso (independientemente de si se crearon bajo demanda o explícitamente) se destruyen automáticamente, de nuevo sin necesidad de hacerlo explícitamente.

Todo ello facilita enormemente la programación de todo tipo de aplicaciones, pero tiene una doble cara: ese dinamismo y automatismo oculta algunos aspectos del funcionamiento de Python, especialmente el funcionamiento de su gestión de memoria. Esto no suele constituir un problema para programadores con cierta experiencia, pero sí puede serlo para quienes se acaban de iniciar en la programación con Python. Para transceder más allá del nivel de iniciación, es necesario adquirir consciencia de *qué sucede realmente* cuando se ejecutan nuestros programas; sólo así podremos tomar las mejores decisiones y emplear las técnicas más adecuadas. Adquirir esa consciencia no implica tener que preocuparnos de hasta el más mínimo detalle; antes al contrario, el dominio de la herramienta y de la técnica nos permitirá emplearlas óptimamente de forma natural. Y cuando nos tropecemos (inevitablemente) con problemas aparentemente inexplicables, saber qué sucede realmente será una ayuda inestimable para solucionarlos.

## Creación y eliminación dinámica de objetos.

Comenzaremos analizando de forma práctica cómo funciona ese dinamismo de Python en lo que respecta a la creación de objetos bajo demanda, así como las condiciones en que los objetos se destruyen automáticamente. Nótese que por *creación* de un objeto entendemos que se reserva memoria para él; y por *destrucción* de un objeto entendemos que la memoria destinada a ese objeto se retorna al *pool* de memoria libre, recuperándola para otros usos. Sin embargo, en condiciones normales lo importante no es cuándo se destruye un objeto, sino cuándo se marca para que sea eventualmente destruido.

### El recolector de basura (*garbage collector*).

Muchos lenguajes que cuentan con gestión dinámica de memoria (no necesariamente lenguajes dinámicos) poseen un componente de su *runtime* llamado **recolector de basura** (*garbage collector*). Su función, como su nombre indica, es recolectar la memoria ocupada por objetos que ya no están en uso con el objetivo de recuperarla (devolviéndola al *pool* de memoria libre) de modo que quede disponible para otros usos, como la creación de nuevos objetos. El recolector de basura puede no estar funcionando continuamente, sino sólo cuando se considera más conveniente.

En el caso de Python, para eliminar automáticamente los objetos que ya no están en uso, utiliza la técnica del *recuento de referencias*. Todo objeto cuenta entre sus atributos con un *contador de referencias*, que almacena en todo momento cuántas referencias existen hacia ese objeto, sea desde variables o desde otros objetos. Cada vez que se crea una referencia a un objeto, se incrementa ese contador; cada vez que se elimina una referencia, se decrementa. Y cuando ese contador llega a cero, el objeto en cuestión **se marca como huérfano** (*orphaned object*), y será eventualmente recolectado por el recolector de basura.


### Objetos referenciados desde variables.

El concepto de variable en Python es muy diferente del que se utiliza en los lenguajes de programación clásicos. En éstos, una variable identifica con su nombre un área de la memoria reservada para almacenar un dato o colección de datos (como un entero, un array, etc.); por tanto, los atributos de una variable en sentido clásico son: el tipo de datos que almacena, su ubicación en memoria, y por supuesto el contenido de dicha ubicación –el valor de la variable–. Por otro lado, en Python una variable tan sólo representa con su nombre una *referencia* a un objeto –en esencia, su ubicación–, siendo dicha referencia el único atributo de la variable. Es decir, una variable de Python no posee más información del objeto al que referencia: ni su tipo/clase, ni su contenido; éstos son atributos del objeto referenciado.

Por ejemplo, dentro de una función escrita en lenguaje C podríamos tener este código:

    int n;    
    n = 5;
    
La primera línea *declara* la variable `n` como de tipo entero, y además la *define*, reservando para ella una ubicación concreta en memoria. En la segunda línea, la asignación copia el valor del literal `5` (que, por su forma, representa el valor 5, de tipo entero) en la ubicación de `n`, por supuesto reemplazando su contenido previo (desconocido en este caso). Obsérvese que ha sido necesario declarar primero el tipo de datos que puede almacenar `n`. Téngase además en cuenta que C es un lenguaje compilado: para poder ejecutar ese programa, primero habrá de ser analizado por el compilador de C, el cual creará un programa ejecutable conteniendo tanto el área de memoria reservada para la variable `n`, como el código que copia el valor entero 5 a esa área de memoria.

En Python, escribiríamos simplemente:

    n = 5

Al ejecutar ese código, el intérprete de Python empezará por crear un nuevo objeto, reservando en ese mismo instante un área de memoria para dicho objeto, e inicializando a continuación sus atributos: su tipo (`int`), su valor (5), y otros atributos, como el número de referencias que existen hacia ese nuevo objeto (inicialmente cero). A continuación realizará la asignación, asociando al identificador `n` la referencia al objeto recién creado (esencialmente, su ubicación en memoria) y, además, incrementará en 1 el contador de referencias de dicho objeto. Obsérvese que no ha habido que declarar la variable `n`, ya que no posee información de tipo de dato, ni tampoco ha sido necesario reservar memoria explícitamente.

Continuando ya con Python, a continuación se crean dos objetos: el número `5` y la cadena `'hola'`:

In [1]:
n = 5
s = 'hola'

Insistimos: no son las asignaciones las que crean ambos objetos, sino su mera aparición como literales en la parte derecha de esas sentencias. Las asignaciones sólo crean sendas referencias desde las variables `n` y `s`.

Se puede ver mejor lo que ha sucedido empleando la función de Python `id`, la cual retorna un número entero que representa la *identidad* de un objeto. En realidad, esa identidad no es otra cosa que la ubicación del objeto en memoria, es decir, la referencia en sí, pero expresada como un `int`.

In [2]:
print(f'n({n})->{id(n)}  s({s})->{id(s)}')

n(5)->4477233984  s(hola)->140452725907376


La *sentencia* `del` permite eliminar una *referencia* a un objeto. En el siguiente ejemplo, primero hacemos que la variable `t` haga referencia al mismo objeto que `s`; pero entonces, eliminamos la referencia `s`:

In [3]:
t = s    # ahora las variables s y t hacen referencia al mismo objeto
print(f's({s})->{id(s)}  t({t})->{id(t)}')
print(f'Adiós a s({s})->{id(s)})')
del s    # eliminamos s, pero el objeto al que hacía referencia sigue existiendo
print(f't({t})->{id(t)}')

s(hola)->140452725907376  t(hola)->140452725907376
Adiós a s(hola)->140452725907376)
t(hola)->140452725907376


Ese objeto `'hola'` sigue existiendo porque aún conserva una referencia (desde la variable `t`). Si ahora eliminamos también esa referencia, el objeto habrá quedado huérfano:

In [4]:
print(f'Adiós a t({t})->{id(t)})')
del t

Adiós a t(hola)->140452725907376)


Desgraciadamente no podemos ver qué ha pasado con ese objeto, porque ya no tenemos forma de acceder a él (hemos perdido todas sus referencias).



### Referencias desde unos objetos a otros.

Por supuesto, no sólo las variables pueden hacer referencia a objetos; también hay objetos que pueden hacer referencia a otros objetos: se trata de aquellos denominados **colecciones**, entre los que se encuentran las listas, tuplas, diccionarios, conjuntos, etc.

A continuación creamos una lista con dos elementos que, en realidad, son el mismo objeto:

In [5]:
l = [1,2,3]
ll = [l,l]
print(ll)

[[1, 2, 3], [1, 2, 3]]


Y aquí tenemos la prueba de que ambos elementos son el mismo objeto:

In [6]:
l.append(4)
print(ll)

[[1, 2, 3, 4], [1, 2, 3, 4]]


---

**INCISO**: En el módulo `sys` se define una función `getrefcount` que, como se puede imaginar, retorna el número de referencias al objeto que se le pasa como parámetro. No obstante, hay que tener en cuenta un par de cosas:

1. Para obtener valores con sentido hemos de emplearlo con solamente con objetos mutables. Ocurre que, con los objetos inmutables, Python puede utilizar técnicas de optimización de recursos (como reutilizar el mismo objeto, por ejemplo), dado que no hay peligro en que estén compartidos (referenciados desde varios sitios). Por tanto, a continuación utilizaremos `getrefcount` con listas, ya que son mutables.
2. `getrefcount` retorna una referencia más de lo que cabe esperar. Esto sucede porque, al pasarle el objeto como parámetro, se crea otra referencia para dicho parámetro.

De modo que importemos ya `getrefcount` desde `sys`:

In [7]:
from sys import getrefcount

y veamos qué nos dice de la listas `l` y `ll`:

In [8]:
print(f'll: {getrefcount(ll)}')
print(f'l: {getrefcount(l)}')

ll: 2
l: 4


Como podemos ver, hay *una* (2-1) referencia al objeto referenciado por `ll`, que es precisamente, la misma variable `ll`. Por otra parte, hay *tres* (4-1) referencias al objeto referenciado por `l`: la misma variable `l` más las dos referencias correspondientes a sendos elementos de `ll`. **FIN DEL INCISO.**

---

Podemos eliminar tranquilamente la variable (referencia) `l` sin que le pase nada a `ll`:

In [9]:
del l
print(ll)

[[1, 2, 3, 4], [1, 2, 3, 4]]


La razón, como hemos visto, es que ese objeto sigue estando referenciado desde cada uno de los elementos de la lista `ll`. Así que:

In [10]:
ll[0].append(5)
print(ll)

[[1, 2, 3, 4, 5], [1, 2, 3, 4, 5]]


¿Podemos comprobar si ha disminuido en 1 el número de referencias tras hacer `del l`? Podemos, pero de este modo:

In [11]:
print(getrefcount(ll[0]))  # (o ll[1], es indiferente)

3


Por supuesto podemos eliminar uno de los elementos de `ll` sin afectar al otro. Aunque ambos elementos hacen referencia al mismo objeto, recuérdese que `del` elimina *la referencia* al objeto, no el objeto referenciado, el cual sólo se destruye cuando queda sin referencias y el recolector de basura recupera la memoria que ocupaba.

In [12]:
del ll[0]
print(ll)
print(getrefcount(ll[0]))

[[1, 2, 3, 4, 5]]
2


Ahora creamos una nueva lista `ll2` que contiene *el mismo* elemento que `ll`:

In [13]:
ll2 = [ll[0], 'hola']
print(ll2)
print(getrefcount(ll[0]))  # (o ll2[0], es lo mismo)

[[1, 2, 3, 4, 5], 'hola']
3


In [14]:
ll[0].append(6)
print(ll,ll2, sep='\n')

[[1, 2, 3, 4, 5, 6]]
[[1, 2, 3, 4, 5, 6], 'hola']


Si ahora eliminanos la variable `ll`, ¿qué pasará? Pues bien: al hacerlo, la lista que estaba referenciada por `ll`, como carece de otras referencias a ella, se destruirá. ¿Pero qué significa destruir una lista? Pues que sus elementos, *que son siempre referencias a objetos*, se eliminan. Pero eso no implica que se eliminen los objetos a los que hacen referencia, si éstos aún conservan otras referencias. Es el caso del único elemento de `ll`, al que también hace referencia el primer elemento de `ll2`. Por tanto, la lista `ll2` quedará incólume, si bien su primer elemento habrá perdido una referencia a él.

In [15]:
del ll
print(ll2)
print(getrefcount(ll2[0]))  # obviamente ya no podemos usar ll[0]

[[1, 2, 3, 4, 5, 6], 'hola']
2


¿Y qué pasa si ahora eliminamos la variable `ll2`? La lista a la que hace referencia carece de otras referencias a ella, así que se destruirá, y con ella sus dos elementos. Pero recordemos que dichos elementos son *referencias* a una lista y una cadena. Estos objetos, a su vez, carecen de otras referencias a ellos, por lo que también se destruirán y el recolector de basura dará cuenta de la memoria que ocupaban.

In [16]:
del ll2
# No podemos usar getrefcount porque ya no disponemos de referencias

**Nótese que no es necesario utilizar `del` para eliminar referencias a objetos.** Por ejemplo, a continuación se muestra un tipo de operación muy habitual:

In [17]:
s = 'otra cadena'
s = s + ' más'
print(s)

otra cadena más


¿Qué ha sucedido ahí?
1. En la primera sentencia, se crea el objeto `'otra cadena'` y, al asignarlo a `s`, se crea también una referencia a dicha cadena.
2. En la segunda sentencia, la expresión de concatenación *crea un nuevo objeto* que contendrá la concatenación de la cadena referenciada por `s` (`'otra cadena'`) con la cadena literal `' más'` (la cual también da lugar a la creación de un nuevo objeto, en este caso efímero).
3. La nueva asignación a `s` reemplaza su referencia a `'otra cadena'` por una nueva referencia a la cadena recién creada por la concatenación. Como consecuencia, el objeto `'otra cadena'` pierde la (única) referencia que tenía (desde `s`), por lo que será destruida.

Otro ejemplo:

In [18]:
l = ['adiós', 'muy', 'buenas']
l[0] = 'hola'
print(l)

['hola', 'muy', 'buenas']


En este caso, al reemplazar `l[0]` por otra cadena, lo que ocurre es que se elimina la referencia al objeto `'adiós'` por otra referencia al objeto `'hola'` y, como consecuencia, la cadena `'adiós'`, pierde la única referencia con que contaba, por lo que será destruida.

### Listas autoreferenciadas

Ya que estamos enredando con listas, viene al caso echar un vistazo a las listas que se autoreferencian, es decir, que son un elemento (o más) de sí mismas. Por ejemplo:

In [19]:
l.append(l)  # Ein?
print(l)
print(l[3])
print(l[3][3][3][3][3])
print(getrefcount(l))

['hola', 'muy', 'buenas', [...]]
['hola', 'muy', 'buenas', [...]]
['hola', 'muy', 'buenas', [...]]
3


Quizá alguien se haya decepcionado porque `getrefcount` no ha reventado, o retornado algo como ¿infinito? Pero no, realmente sólo hay dos referencias a esa lista: una desde la variable `l`, y otra desde el último elemento de sí misma.

Por supuesto también podemos crear también toda clase de referencias cruzadas (o cíclicas, en general) con varias listas. Ejemplo:

In [20]:
l2 = [1, 2, 3]
l[3] = l2
l2.append(l)
print(f'l  = {l}')
print(f'l2 = {l2}')

l  = ['hola', 'muy', 'buenas', [1, 2, 3, [...]]]
l2 = [1, 2, 3, ['hola', 'muy', 'buenas', [...]]]


Obviamente se pueden montar estructuras similares usando diccionarios, por ejemplo.


### Las referencias en el paso de parámetros a funciones (y retorno de valores).

En Python también se usan las referencias a objetos como mecanismo de paso de parámetros a funciones. En los lenguajes clásicos se emplean mecanismos como el paso por valor (se copia el valor del parámetro en la llamada al parámetro formal de la función), o el paso por referencia (sólo válido para pasar variables, se copia la dirección de la variable). El mecanismo que utiliza Python se denomina formalmente **paso de parámetros por referencia a objeto**.

Como mejor se entiende esto es con un ejemplo sencillo:

In [21]:
def f(l):
    print(f'id(l) = {id(l)}')
    return l

lista = [1,2,3]
print(f'id(lista) = {id(lista)}')
retorno = f(lista)
print(f'id(retorno) = {id(retorno)}')

id(lista) = 140452726426432
id(l) = 140452726426432
id(retorno) = 140452726426432


Obsérvese que `lista` (fuera de la función), `l` (el parámetro formal de la función) y `retorno` (el valor que ha retornado la función) *hacen referencia al mismo objeto*. La razón es que al invocar `f(lista)` simplemente ha copiado la referencia `lista` al parámetro formal `l` (exactamente igual que si se hubiese asignado `l = lista`). Por otra parte, la sentencia `return l` hace que el valor de retorno de la función sea la referencia `l`, la cual se asigna finalmente a la variable `retorno`.

¿Y qué sucede con el contador de referencias de ese objeto? Veamos:

In [22]:
def f(l):
    print(f'Dentro: refcount(l[{id(l)}]) = {getrefcount(l)}')
    return l

lista = [1,2,3]
print(f'Antes: refcount(lista[{id(lista)}]) = {getrefcount(lista)}')
retorno = f(lista)
print(f'Después: refcount(retorno[{id(retorno)}]) = {getrefcount(retorno)}')

Antes: refcount(lista[140452726391680]) = 2
Dentro: refcount(l[140452726391680]) = 4
Después: refcount(retorno[140452726391680]) = 3


Recuérdese que `getrefcount` retorna una referencia más. Obviamente, antes de llamar a la función hay una sola referencia al objeto (desde `lista`). Al llamar a la función, se crean dos nuevas referencias: una corresponde con seguridad al parámetro `l`; la otra seguramente se crea como parte del mecanismo de llamada a la función. Una vez que se retorna de la función, vemos que hay dos referencias: una sigue siendo `lista`, y la otra es `retorno`. El caso es que las referencias creadas al llamar a la función desaparecen tras su retorno.

¿Y qué sucede si escribimos una expresión como parámetro? Por ejemplo: `f(lista+lista)`. Python simplemente crea un objeto nuevo automáticamente para almacenar el resultado de la expresión `lista+lista` (recordemos la naturaleza dinámica de Python), de modo que lo que pasa como parámetro es la referencia a ese objeto:

In [23]:
lista = [1,2,3]
print(f'Antes: refcount(lista[{id(lista)}]) = {getrefcount(lista)}')
retorno = f(lista+lista)
print(f'Después: refcount(retorno[{id(retorno)}]) = {getrefcount(retorno)}')
print(retorno)

Antes: refcount(lista[140452726139456]) = 2
Dentro: refcount(l[140452726393280]) = 3
Después: refcount(retorno[140452726393280]) = 2
[1, 2, 3, 1, 2, 3]


Podemos ver que `l` (ya dentro de la función) y por tanto `retorno` referencian un objeto diferente del referenciado por `lista`. Ese nuevo objeto es, como se puede ver, el resultado de evaluar la expresión `lista+lista`.

## Constructores.

Un **constructor** es una función (o aparenta serlo) cuya misión es crear un nuevo objeto perteneciente a una **clase**, retornando aquél la referencia al objeto recién creado. En la terminología OOP (*Object-Oriented Programming*), se denomina **instancia** (*instance*) a cada uno de los objetos de una determinada clase. Así, por ejemplo, el objeto `'hola'` es una instancia de la clase `str`. A la acción de crear un nuevo objeto de una determinada clase se le llama *instanciar* (la clase). La RAE no admite reclamaciones al respecto. :)

Como ya hemos visto, en Python no siempre es necesario utilizar un constructor para crear nuevos objetos, gracias a su naturaleza dinámica. No obstante, el uso de constructores constituye la forma ortodoxa de instanciar clases.

A estas alturas, ya hemos utilizado constructores en muchísimas ocasiones, aunque sin haber sido plenamente conscientes de ello. Algunos ejemplos:

    n = int(input('Dime un número: '))
    x = float(m)
    s = str(z/2)
    c = complex(a,b)
    l = list(range(10))
    d = dict(a=1, b=2, c=3)

Los constructores se denominan siempre con el nombre de la clase que instancian: así, la función `int` es el constructor de la clase `int`, etc.

Lógicamente, un constructor se encarga de crear (reservar memoria para) el nuevo objeto. Generalmente, también *inicializa* el objeto recién creado, bien a partir de parámetros que se le suministren, o bien con un valor predeterminado, en ausencia de parámetros.

Conviene señalar que en ocasiones la creación explícita de objetos no se realiza mediante el uso directo de constructores. Un ejemplo bien conocido es el de los objetos tipo archivo, que se crean mediante la función `open`. Evidentemente, `open` utiliza internamente el constructor de archivos, pero dado que abrir un archivo implica muchas más operaciones, entre ellas las de acceso al sistema de archivos, resulta mucho más sencillo y práctico contar con una función como `open` que se encarga de todo esto, y retorna el objeto archivo ya preparado para su uso.

Otra particularidad notable son las funciones, que por supuesto también son objetos, pero que se crean empleando la sintaxis `def` (que, en realidad, es otra forma de invocar al constructor de la clase `function`).

Muchos módulos de Python definen clases, con sus constructores, como es lógico:

In [24]:
from fractions import Fraction

f1 = Fraction()
f2 = Fraction('2/3')
f3 = Fraction(0.75)

print(f1, f2, f3, f2+f3)

0 2/3 3/4 17/12


Por supuesto nosotros podemos definir nuestras propias clases. Esto se verá con detalle más adelante en la asignatura, pero presentamos aquí un sencillo ejemplo para abrir boca:

In [25]:
class Estudiante():
    def __init__(self, nombre, apellido, dni, *asignaturas):
        self.nombre = nombre
        self.apellido = apellido
        self.dni = dni
        self.asignaturas = set(asignaturas)
    
    def matricula(self, *asignaturas):
        self.asignaturas.update(asignaturas)
    
    def desmatricula(self, *asignaturas):
        self.asignaturas.difference_update(asignaturas)
        
    def esta_matriculado_en(self, asignatura):
        return asignatura in self.asignaturas
    
    def __repr__(self):
        return f'Estudiante({self.nombre!r}, {self.apellido!r}, {self.dni!r}, {self.asignaturas!r})'

Hemos definido una clase `Estudiante` en cuyas instancias se almacena la identidad del estudiante (nombre, apellido y DNI como `str`) y las asignaturas de las que está matriculado como conjunto (clase `set` de Python). Además, se definen unos métodos que permiten matricular y desmatricular de una o más asignaturas, y consultar si está matriculado de una asignatura.

A continuación usaremos el constructor para crear dos estudiantes y enredar con sus matrículas:

In [26]:
# Conjunto de asignaturas comunes a 1º de MAT e IE (BIO es un error deliberado):
asig_1MATIE  = {'FIS.G', 'CAL.D', 'ALG.L', 'I.COMP', 'F.PROG', 'BIO'}
                      
# Creamos dos estudiantes, ya matriculados de las asignaturas comunes:
e1 = Estudiante('Rebeca', 'Millán', '11222000Q', *asig_1MATIE)
e2 = Estudiante('Aureliano', 'Buendía', '44555000W', *asig_1MATIE)

print(e1, e2, sep='\n')

# Ahora matriculamos a e1 de las dos asignaturas de 1º de MAT, y a e2 de las dos de 1º de IE:
print(f'** Corregimos matrículas de {e1.nombre} y {e2.nombre}:')

e1.matricula('EST.D', 'MAT.B')
e2.matricula('T.EXPI', 'QUIM.I')
for e in e1,e2:
    e.desmatricula('BIO')

print(e1, e2, sep='\n')

Estudiante('Rebeca', 'Millán', '11222000Q', {'F.PROG', 'FIS.G', 'ALG.L', 'CAL.D', 'BIO', 'I.COMP'})
Estudiante('Aureliano', 'Buendía', '44555000W', {'F.PROG', 'FIS.G', 'ALG.L', 'CAL.D', 'BIO', 'I.COMP'})
** Corregimos matrículas de Rebeca y Aureliano:
Estudiante('Rebeca', 'Millán', '11222000Q', {'F.PROG', 'EST.D', 'MAT.B', 'FIS.G', 'ALG.L', 'CAL.D', 'I.COMP'})
Estudiante('Aureliano', 'Buendía', '44555000W', {'F.PROG', 'T.EXPI', 'FIS.G', 'ALG.L', 'CAL.D', 'QUIM.I', 'I.COMP'})


Como se puede ver, no hay que preocuparse de la gestión de memoria al instanciar la clase `Estudiante`: Python se encarga de todo lo necesario: simplemente invocamos su función constructora, y automáticamente se crea el objeto. Obsérvese que en la definición de la clase `Estudiante` se incluye un método `__init__`, el cual es invocado automáticamente al instanciar la clase. Sus parámetros se emplean para inicializar el contenido (los *atributos*) del objeto creado: en este caso el nombre, apellido, DNI (obligatorios), y también se inicializa el conjunto de asignaturas de las que se halla matriculado el estudiante. Si no se especifica ninguna en el momento de usar el constructor, se inicializará con un conjunto de asignaturas vacío.

El método `__repr__` se invocará automáticamente al llamar a la función `repr` pasándole como parámetro un objeto de clase `Estudiante`. Lo hemos añadido para poder mostrar en pantalla las instancias de `Estudiante` simplemente con un `print`.

(Nótese que la cadena que retorna el método `__repr__` de la clase `Estudiante` se ha diseñado para que tenga exactamente la misma forma que la invocación del constructor para crear una instancia idéntica. Eso significa que podríamos hacer, por ejemplo, `e3 = eval(repr(e1))` para crear una copia de `e1` y referenciarla con la variable `e3`.)

En cuanto a esos parámetros que aparecen precedidos de `*`, se trata de algo que veremos en otro seminario. Baste decir que se utiliza para poder pasar un número variable de asignaturas como parámetros, bien especifícandolas una por una, o bien agrupadas en un conjunto, por ejemplo.

## Copia superficial y en profundidad (*shallow copy* y *deep copy*).

La manipulación de objetos mediante referencias es un mecanismo sencillo y eficiente. Sin embargo, en ocasiones puede tener efectos inesperados para quienes se inician en Python; o incluso para programadores experimentados, pero habituados a los lenguajes clásicos.

Uno de esos efectos es que no resulta trivial la copia de objetos compuestos. Este problema se suele abordar en cuanto se comienza a trabajar con listas. Si queremos crear una lista `l2` como copia de una lista `l1` ya existente, sabemos que esto no funciona:

    l1 = [1, 2, 3]
    l2 = l1

puesto que con `l2 = l1` tan sólo estamos copiando la referencia a la lista, y como resultado `l2` y `l1` representarán el mismo objeto. Ya sabemos que una solución es emplear una expresión de indexación para generar un nuevo objeto que contenga los mismos elementos que la lista original:

    l2 = l1[:]

Pero ¿qué sucede si la lista `l1` tiene como elemento otra lista? Veamos qué sucede aquí:

In [27]:
l1 = [1, 2, ['a', 'b', 'c']]
l2 = l1[:]
l2.append(666)
l2[2][0] = 'ZZZ'
print(f'l1={l1}\nl2={l2}')

l1=[1, 2, ['ZZZ', 'b', 'c']]
l2=[1, 2, ['ZZZ', 'b', 'c'], 666]


Mediante la expresión de indexación hemos realizado lo que se denomina una **copia superficial** (*shallow copy*) de la lista original, de modo que podemos añadir, eliminar o reemplazar elementos de una de las copias sin afectar a la otra. Pero, como los elementos de una lista son referencias a objetos, `l1[2]` y `l2[2]` hacen referencia al mismo objeto. Entonces, ¿cómo podemos realizar una **copia en profundidad** (*deep copy*) de un objeto compuesto, creando copias de todos los objetos que contiene?

Una posible solución es ésta:

In [28]:
l1 = [1, 2, ['a', 'b', 'c']]
l2 = eval(repr(l1))   # <--- ¡TRUCO!
l2.append(666)
l2[2][0] = 'ZZZ'
print(f'l1={l1}\nl2={l2}')

l1=[1, 2, ['a', 'b', 'c']]
l2=[1, 2, ['ZZZ', 'b', 'c'], 666]


El *truco* se basa en esta información que proporciona `help(repr)`:

    repr(obj, /)
        Return the canonical string representation of the object.
    
        For many object types, including most builtins, eval(repr(obj)) == obj.

Pero obsérvese que no se garantiza que esto se verifique para *todos* los tipos de objetos. Es más, en el caso de las clases que definamos nosotros, sería responsabilidad nuestra (y en todo caso opcional) diseñar su método `__repr__` para que cumpla esa propiedad (precisamente, es lo que hemos hecho en la definición de la clase `Estudiante` mostrada arriba).

Además, no siempre será posible: durante la ejecución de un programa pueden construirse objetos compuestos de complejidad indeterminada, de modo que `repr` ya no podrá generar una representación canónica de ellos procesable por `eval`.

Afortunadamente, existe una solución: emplear las funciones definidas en el [módulo `copy` de la biblioteca estándar de Python](https://docs.python.org/3/library/copy.html).

In [29]:
import copy

l1 = [1, 2, ['a', 'b', 'c']]
l2 = copy.deepcopy(l1)
l2.append(666)
l2[2][0] = 'ZZZ'
print(f'l1={l1}\nl2={l2}')

l1=[1, 2, ['a', 'b', 'c']]
l2=[1, 2, ['ZZZ', 'b', 'c'], 666]


El módulo `copy` ofrece dos funciones: `copy.deepcopy`, que realiza una copia *en profundidad* del objeto que se le pasa como parámetro; y `copy.copy`, que realiza una copia *superficial*. Téngase en cuenta que, si bien es posible realizar copias superficiales de listas mediante la expresión de indexación `[:]`, y de diccionarios mediante su método `dict.copy`, necesitaremos `copy.copy` para realizar copias superficiales de objetos de otras clases.

Quizá en este punto alguien se pregunte qué sucede con las cadenas y las tuplas. Bien, recuérdese que en ambos casos se trata de objetos inmutables, por lo que no tiene ningún sentido realizar copias superficiales de ellos: ¿para qué vamos a hacer una copia superficial, si no es posible modificar el original? Sin embargo, sí que puede tener sentido realizar una copia en profundidad de las tuplas, dado que éstas pueden contener elementos mutables a cualquier nivel de profundidad:

In [30]:
t1 = (1, 2, ['a', 'b', 'c'])
t2 = t1[:]    # copia superficial
t2[2][0] = 'ZZZ'
print(f't1={t1}\nt2={t2}\n')

t1 = (1, 2, ['a', 'b', 'c'])
t2 = copy.deepcopy(t1)    # copia en profundidad
t2[2][0] = 'ZZZ'
print(f't1={t1}\nt2={t2}')

t1=(1, 2, ['ZZZ', 'b', 'c'])
t2=(1, 2, ['ZZZ', 'b', 'c'])

t1=(1, 2, ['a', 'b', 'c'])
t2=(1, 2, ['ZZZ', 'b', 'c'])


### El módulo `pickle`.

En relación a las copias en profundidad, resulta conveniente mencionar [el módulo `pickle` de la biblioteca estándar de Python](https://docs.python.org/3/library/pickle.html). Este módulo permite “serializar” un objeto, generando a partir de él un *stream* que se almacena en un archivo. Lógicamente, también permite realizar el proceso inverso, es decir, crear un nuevo objeto a partir de lo previamente serializado y guardado en un archivo. Para ello el módulo `pickle` ofrece las funciones `pickle.dump` y `pickle.load`. Resulta de utilidad para guardar estructuras de datos en archivos con el objetivo de poder recuperarlas más tarde exactamente en la misma forma.

La función `pickle.dump` realiza un recorrido en profundidad del objeto a serializar y guardar en un archivo. Cuando con posterioridad se utiliza `pickle.load` para recuperar el contenido del archivo, se crea un nuevo objeto que es una copia del original.

Hay que tener en cuenta que no todos los objetos son *pickables* (véase la documentación del módulo). Por otra parte, el formato de serialización (denominado `protocolo`) es propio de Python, e incluso puede haber incompatibilidades de protocolo entre distintas versiones de Python.

Veamos un ejemplo sencillo:

In [31]:
import pickle

d = dict(a=[1,2,3], b=['hola', 'adios'], c=666, d=True)
print('d original:', d)

with open('d.pickle', 'wb') as f:     # el archivo almacena datos binarios, no texto
    pickle.dump(d,f)                  # serializamos y guardamos d en el archivo

# Más tarde, incluso desde otro programa que manipule el mismo tipo de estructura de datos…

with open('d.pickle', 'rb') as f:
    new_d = pickle.load(f)            # recuperamos el objeto desde el archivo
    
print('d recuperado:', new_d)

d original: {'a': [1, 2, 3], 'b': ['hola', 'adios'], 'c': 666, 'd': True}
d recuperado: {'a': [1, 2, 3], 'b': ['hola', 'adios'], 'c': 666, 'd': True}


### El módulo `json`.

Conceptualmente similar al módulo `pickle`, [el módulo `json` de la biblioteca estándar de Python](https://docs.python.org/3/library/json.html) ofrece las funciones `json.dump` y `json.load` para codificar y almacenar un objeto en un archivo en formato JSON, y viceversa. La ventaja del formato JSON es que está diseñado para intercambiar datos, pero tiene como desventaja que los tipos de objetos de admite están mucho más limitados.

Veamos un ejemplo:

In [32]:
import json

d = dict(a=[1,2,3], b=['hola', 'adios'], c=666, d=True)
print('d original:', d)

with open('d.json', 'w', encoding='utf-8') as f:     # JSON es un formato de texto, en UTF-8
    json.dump(d,f)                                   # codificamos y guardamos d en el archivo

# Más tarde, incluso desde otro programa que manipule el mismo tipo de estructura de datos…

with open('d.json', encoding='utf-8') as f:
    new_d = json.load(f)                             # recuperamos el objeto desde el archivo
    
print('d recuperado:', new_d)

d original: {'a': [1, 2, 3], 'b': ['hola', 'adios'], 'c': 666, 'd': True}
d recuperado: {'a': [1, 2, 3], 'b': ['hola', 'adios'], 'c': 666, 'd': True}


Y esto es lo que se guarda en el archivo `d.json`:

    {"a": [1, 2, 3], "b": ["hola", "adios"], "c": 666, "d": true}